In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from statsmodels.stats.proportion import proportion_confint

# Корреляционный анализ
## Годовая смертность
Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

In [2]:
df = pd.read_csv('water.txt', sep = '\t')
df.head(10)

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18
5,North,Bolton,1558,10
6,North,Bootle,1807,15
7,South,Bournemouth,1299,78
8,North,Bradford,1637,10
9,South,Brighton,1359,84


### Смертность и жесткость воды
Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

In [3]:
mortality_corr = df.corr(method = 'pearson')
np.round(mortality_corr.iloc[0, 1], 4)

-0.6548

В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки. 

In [4]:
mortality_corr = df.corr(method = 'spearman')
np.round(mortality_corr.iloc[0, 1], 4)

-0.6317

### Корреляция по северным и южным городам
Сохраняется ли связь между признаками, если разбить выборку на северные и южные города? Посчитайте значения корреляции Пирсона между средней годовой смертностью и жёсткостью воды в каждой из двух подвыборок, введите наименьшее по модулю из двух значений, округлив его до четырёх знаков после десятичной точки.

In [5]:
df_south = df[df['location'] == 'South']
mortality_south_corr = df_south.corr(method = 'pearson')
np.abs(np.round(mortality_south_corr.iloc[0, 1], 4))

0.6022

In [6]:
df_north = df[df['location'] == 'North']
mortality_north_corr = df_north.corr(method = 'pearson')
np.abs(np.round(mortality_north_corr.iloc[0, 1], 4))

0.3686

## Кто больше пьёт: мужчины или женщины?
Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.

In [7]:
a = 718
b = 203
c = 515
d = 239

mcc = (a*d - b*c) / np.sqrt((a+b)*(a+c)*(b+d)*(c+d))

print(np.round(mcc, 3))

0.109


В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию scipy.stats.chi2_contingency. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

In [8]:
sp.stats.chi2_contingency(np.array([[a, b], [c, d]]))[1]

1.0558987006638725e-05

В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.

In [9]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = sp.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [10]:
women = [0] * a + [1] * b
men = [0] * c + [1] * d

print("confidence interval: [%f, %f]" % proportions_confint_diff_ind(men, women))

confidence interval: [0.053905, 0.139222]


Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу. Введите номер первой значащей цифры (например, если вы получили $
5.5\times10^{-8}$, нужно ввести 8).

In [11]:
sp.stats.ttest_ind(men, women)

Ttest_indResult(statistic=4.485175262463444, pvalue=7.78139336142543e-06)

## Довольный и при деньгах
Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"
Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

' | Не доволен | Более или менее | Доволен
--- | --- | ---| ---
Не очень счастлив | 197 | 111 | 33
Достаточно счастлив	| 382| 685| 331
Очень счастлив	| 110| 342| 333

In [12]:
HappinessTable = np.array([[197, 111, 33], [382, 685, 331], [110, 342, 333]])
result = sp.stats.chi2_contingency(HappinessTable)
np.round(result[0], 4)

293.6831

На данных из предыдущего вопроса посчитайте значение достигаемого уровня значимости. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

In [13]:
result[1]

2.4964299580093467e-62

Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков? Округлите ответ до четырёх знаков после десятичной точки.
$$\varphi_c(X_1, X_2) = \sqrt{\frac{\chi(X_1, X_2)}{n(min(K_1,K_2) - 1)}}$$

In [14]:
chi = result[0]
n = np.sum(HappinessTable)
np.round(np.sqrt(chi / (n * (3 - 1))), 4)

0.2412

## Результат
Был закреплен материал из лекций. В особенности метод V Крамера.